In [142]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [143]:
dataset = pd.read_csv('Hotdogs.csv')


In [144]:
dataset.head()

,game,day,section,vendor,hot_dogs_sold
0,1,2,1,6,145
1,1,2,2,10,121
2,1,2,3,21,56
3,1,2,4,8,123
4,1,2,5,4,100


In [158]:
dataset.describe()

,game,day,section,vendor,hot_dogs_sold,Gfactor,Dfactor,Sfactor,Factored_hot_dogs_sold
count,1588.000000,1588.000000,1588.000000,1588.000000,1588.000000,1588.000000,1588.000000,1588.000000,1588.000000
mean,40.943955,3.968514,10.539043,11.928212,135.066121,0.746016,0.815015,0.818087,275.397768
std,23.339337,1.974383,5.767966,6.949717,65.781397,0.112655,0.099154,0.072526,131.209141
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.532450,0.683104,0.673972,1.960650
25%,21.000000,2.000000,6.000000,6.000000,88.000000,0.663963,0.735000,0.780209,183.823504
50%,41.000000,4.000000,11.000000,12.000000,129.000000,0.734880,0.780252,0.808222,268.285425
75%,61.000000,6.000000,16.000000,17.000000,178.000000,0.810477,0.915260,0.838234,355.436494
max,81.000000,7.000000,20.000000,30.000000,432.000000,1.000000,1.000000,1.000000,954.142074


Romoving rows where hot_dogs_sold is 0, as that section might be close and does not affect the ability of the vendor

In [146]:
dataset = dataset[dataset.hot_dogs_sold != 0]


Seperating the game, day and section data to remove their affect on the hotdogs sold and normalizing their affect

In [147]:
gm = dataset.groupby(['game']).mean()
gm.drop(columns=['day', 'section','vendor'], axis=1, inplace=True)
gm=gm/gm.max()
gm= gm.rename(columns={'game': 'game', 'hot_dogs_sold': 'Gfactor'})
gm.head()

,Gfactor
game,
1,0.743993
2,0.668600
3,0.704952
4,0.784866
5,0.812803


In [148]:
day = dataset.groupby(['day']).mean()
day.drop(columns=['game', 'section','vendor'], axis=1, inplace=True)
day=day/day.max()
day= day.rename(columns={'day': 'day', 'hot_dogs_sold': 'Dfactor'})
day

,Dfactor
day,
1,0.683104
2,0.775961
3,0.735000
4,0.780252
5,0.829860
6,0.915260
7,1.000000


In [149]:
section = dataset.groupby(['section']).mean()
section.drop(columns=['game', 'day','vendor'], axis=1, inplace=True)
section=section/section.max()
section= section.rename(columns={'section': 'section', 'hot_dogs_sold': 'Sfactor'})
section.head()

,Sfactor
section,
1,0.979550
2,0.780209
3,0.779073
4,0.755719
5,0.788523


In [150]:
dataset = dataset.merge(gm, on='game')
dataset = dataset.merge(day, on='day')
dataset = dataset.merge(section, on='section')
dataset.head()

,game,day,section,vendor,hot_dogs_sold,Gfactor,Dfactor,Sfactor
0,1,2,1,6,145,0.743993,0.775961,0.97955
1,15,2,1,2,184,0.666114,0.775961,0.97955
2,22,2,1,11,239,0.686827,0.775961,0.97955
3,29,2,1,1,297,0.888429,0.775961,0.97955
4,36,2,1,7,95,0.734880,0.775961,0.97955


Disregarding the affect of other factors( game, day, section) for the number of hot dogs sold

In [151]:
dataset['Factored_hot_dogs_sold'] = dataset.hot_dogs_sold/(dataset.Gfactor * dataset.Dfactor * dataset.Sfactor )
dataset.head()

,game,day,section,vendor,hot_dogs_sold,Gfactor,Dfactor,Sfactor,Factored_hot_dogs_sold
0,1,2,1,6,145,0.743993,0.775961,0.97955,256.408470
1,15,2,1,2,184,0.666114,0.775961,0.97955,363.414688
2,22,2,1,11,239,0.686827,0.775961,0.97955,457.808732
3,29,2,1,1,297,0.888429,0.775961,0.97955,439.812274
4,36,2,1,7,95,0.734880,0.775961,0.97955,170.075081


In [152]:
dataset1 = dataset.groupby(['vendor']).mean()
dataset1.head()

,game,day,section,hot_dogs_sold,Gfactor,Dfactor,Sfactor,Factored_hot_dogs_sold
vendor,,,,,,,,
1,41.760563,3.915493,10.521127,123.661972,0.742604,0.813317,0.814093,248.782321
2,42.984848,4.000000,10.136364,142.954545,0.752765,0.818200,0.826306,280.441591
3,40.295775,4.028169,10.239437,151.957746,0.744397,0.816605,0.808818,322.855003
4,40.540541,3.986486,9.148649,113.189189,0.745268,0.814520,0.818932,230.689746
5,41.191781,4.027397,10.712329,142.178082,0.748599,0.819292,0.805376,292.515303


Sorting the vendors based on the effective average number of hot dogs sold 

In [154]:
dataset2 = dataset1.drop(['game','day','section','hot_dogs_sold','Gfactor','Dfactor','Sfactor'], axis=1)
dataset2 = dataset2.sort_values(by=['Factored_hot_dogs_sold'], ascending=False)
dataset2.head()

,Factored_hot_dogs_sold
vendor,
29,392.722348
20,344.911973
21,328.627521
27,323.931225
3,322.855003


In [157]:
from google.colab import files
dataset2.to_csv('Ranked_vendors.csv') 
files.download('Ranked_vendors.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>